In [27]:
patient = 'P6'

In [28]:
segments = "../ascat/ascat_outputs/%s_logr_segments.csv" % patient
snpdata = "../ascat/ascat_outputs/%s_snpdata.csv" % patient
bafdata = "../ascat/ascat_inputs/%s_ascat_baf.txt" % patient
rdrdata = "../ascat/ascat_inputs/%s_ascat_logr.txt" % patient

In [29]:
# output files
output_file = "%s_cnaviz_input.txt" % patient

In [30]:
inputdata = "cnaviz/data/Casasent2018/%s/results/best.bbc.ucn" % patient

In [31]:
# chrm -> bins
segment_dict = dict()
# HARDCODED to sample names
segment_dict['DCIS'] = []
segment_dict['INV'] = []

last_dcis_val = ''
last_inv_val = ''

segidx_dcis = 0
segidx_inv = 0

with open(segments, "r") as r:
    lines = r.readlines()
    for i, line in enumerate(lines[1:]):
        dcis, inv = line.rstrip().split(',')
        
        if last_dcis_val == '':
            last_dcis_val = dcis
        if last_inv_val == '':
            last_inv_val = inv
        
        if dcis != last_dcis_val:
            segidx_dcis += 1
            last_dcis_val = dcis
        if inv != last_inv_val:
            segidx_inv += 1
            last_inv_val = inv
            
        segment_dict['DCIS'].append(segidx_dcis)
        segment_dict['INV'].append(segidx_inv)

In [32]:
# num clusters
segidx_dcis, segidx_inv

(27, 27)

In [33]:
len(segment_dict['DCIS'])

6672

In [34]:
chrm_dict = dict()
bin_idx = 0
with open(snpdata, "r") as r:
    lines = r.readlines()
    for i, line in enumerate(lines[1:]):
        chrm, pos = line.rstrip().split(',')
        if chrm not in chrm_dict.keys():
            chrm_dict[chrm] = dict()
            bin_idx = 0
        chrm_dict[chrm][bin_idx] = dict()
        chrm_dict[chrm][bin_idx]['DCIS'] = segment_dict['DCIS'][i]
        chrm_dict[chrm][bin_idx]['INV'] = segment_dict['INV'][i]
        bin_idx += 1

In [35]:
chrm_dict.keys()

dict_keys(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22'])

In [36]:
# keeps which bin we've seen so far 
chrm_count_dict = dict()
chrm_count_dict['DCIS'] = dict()
chrm_count_dict['INV'] = dict()

with open(inputdata, "r") as r:
    with open(output_file, "w+") as w:
        lines = r.readlines()
        for _, line in enumerate(lines):
            if line[0] == 'c': 
                vals = line.split('\t')
                chrm = vals[0][3:]
                samplename = vals[3]
                
                if chrm not in chrm_count_dict[samplename].keys():
                    chrm_count_dict[samplename][chrm] = 0
                bin_idx = chrm_count_dict[samplename][chrm]
                
                #print(samplename, bin_idx, chrm)
                #print(chrm_dict[str(chrm)][bin_idx], bin_idx, samplename)
                vals[10] = str(chrm_dict[str(chrm)][bin_idx][samplename])
                chrm_count_dict[samplename][chrm] += 1
                
                line = '\t'.join(vals)
            w.write(line)

# Difficult Way

In [ ]:
cluster_idx = 0
snp_file_idx = 0
cluster_set = set()
progress_cluster = False
with open(segments, "r") as r:
    lines = r.readlines()
    
    # create seen sets
    sample_seen_dict = dict()
    for sample_idx in range(len(lines[0].split(','))):
        sample_seen_dict[sample_idx] = set()
        
    for line in lines[1:]:
        seg_values = line.rstrip().split(',')
        for seg_value in seg_values:
            if seg_value not in sample_seen_dict:
                sample_seen_dict = set()
                sample_seen_dict.add(seg_value)
                progress_cluster = True
        if progress_cluster:
            cluster_idx += 1
        
        chrm, pos = snpfile_dict[snp_file_idx]
        mysnpdict = chrm_dict1[chrm][pos]
        mysnpdict['cluster'] = cluster_idx
        cluster_set.add(cluster_idx)
        snp_file_idx += 1
print(cluster_idx, "clusters assigned.")

In [111]:
import numpy as np

# chrm_dict1[chrm]
    # -> snp_dict[snppos]
            # -> sample[samplename]
                    # -> rdr value
                    # -> baf value
            # -> cluster
                    
# snp_file_idx[idx]
    # -> chrm, snppos
    
# Setting up chrm_dict1 and snp_dict
chrm_dict1 = dict()
snpfile_dict = dict()
snp_file_idx = 0
with open(snpdata, "r") as r:
    lines = r.readlines()
    for line in lines[1:]:
        chrm, pos = line.rstrip().split(',')
        snpfile_dict[snp_file_idx] = (chrm, pos)
        
        if chrm not in chrm_dict1.keys():
            chrm_dict1[chrm] = dict()
        snp_dict = chrm_dict1[chrm]
        if pos not in snp_dict.keys():
            snp_dict[pos] = dict()
        mysnpdict = snp_dict[pos]
        mysnpdict['file_idx'] = snp_file_idx # starts at 0
        snp_file_idx += 1

# For each snp, add the cluster assignment
cluster_idx = 0
snp_file_idx = 0
cluster_set = set()
progress_cluster = False
with open(segments, "r") as r:
    lines = r.readlines()
    
    # create seen sets
    sample_seen_dict = dict()
    for sample_idx in range(len(lines[0].split(','))):
        sample_seen_dict[sample_idx] = set()
        
    for line in lines[1:]:
        seg_values = line.rstrip().split(',')
        for seg_value in seg_values:
            if seg_value not in sample_seen_dict:
                sample_seen_dict = set()
                sample_seen_dict.add(seg_value)
                progress_cluster = True
        if progress_cluster:
            cluster_idx += 1
        
        chrm, pos = snpfile_dict[snp_file_idx]
        mysnpdict = chrm_dict1[chrm][pos]
        mysnpdict['cluster'] = cluster_idx
        cluster_set.add(cluster_idx)
        snp_file_idx += 1
print(cluster_idx, "clusters assigned.")
        
# For each snp, add the rdr and baf values for each sample
# HARDCODED with sample names
snp_file_idx = 0
with open(bafdata, "r") as r:
    lines = r.readlines()
    for line in lines[1:]:
        snpname, chrm, pos, s1, s2 = line.split()
        mysnpdict = chrm_dict1[chrm][pos]
        if 'samples' not in mysnpdict.keys():
            mysnpdict['samples'] = dict()
        mysnpdict['samples']['DCIS'] = [float(s1)] # first val with be baf
        mysnpdict['samples']['INV'] = [float(s2)]
    snp_file_idx += 1

snp_file_idx = 0
with open(rdrdata, "r") as r:
    lines = r.readlines()
    for line in lines[1:]:
        snpname, chrm, pos, s1, s2 = line.split()
        mysnpdict = chrm_dict1[chrm][pos]
        if 'samples' not in mysnpdict.keys():
            mysnpdict['samples'] = dict()
        mysnpdict['samples']['DCIS'].append(np.exp(float(s1))) # second val with be rdr (NOT LOG)
        mysnpdict['samples']['INV'].append(np.exp(float(s2)))
    snp_file_idx += 1

6448 clusters assigned.


In [112]:
len(cluster_set)

6448

In [125]:
def find_snps(d, s, w):
    # HARDCODED samplenames
    num_snps = 0
    sample_dict = dict()
    sample_dict['DCIS'] = dict()
    sample_dict["DCIS"]['RDR'] = []
    sample_dict["DCIS"]['BAF'] = []

    sample_dict['INV'] = dict()
    sample_dict["INV"]['RDR'] = []
    sample_dict["INV"]['BAF'] = []
    
    cluster_list = []
    
    for snppos in d.keys():
        if s <= int(snppos) and int(snppos) < s + w: 
            num_snps += 1
            print(s)
            print("\t", snppos, d[snppos])
            print(s + w)
            baf, rdr = d[snppos]['samples']['DCIS']
            sample_dict["DCIS"]["RDR"].append(rdr)
            sample_dict["DCIS"]["BAF"].append(baf)

            baf, rdr = d[snppos]['samples']['INV']
            sample_dict["INV"]["RDR"].append(rdr)
            sample_dict["INV"]["BAF"].append(baf)
            
            cluster_list.append(d[snppos]['cluster'])
    
    if num_snps == 0:
        return None
    #print(num_snps)
    
    rv_sample = dict()
    rv_sample["INV"] = dict()
    rv_sample["INV"]["RDR"] = np.mean(sample_dict["INV"]["RDR"])
    rv_sample["INV"]["BAF"] = np.mean(sample_dict["INV"]["BAF"])
    
    rv_sample["DCIS"] = dict()
    rv_sample["DCIS"]["RDR"] = np.mean(sample_dict["DCIS"]["RDR"])
    rv_sample["DCIS"]["BAF"] = np.mean(sample_dict["DCIS"]["BAF"])
    
    cluster = max(cluster_list, key=cluster_list.count)
    return cluster, num_snps, rv_sample
            

In [126]:
# chrm_dict2[chrm]
    # -> bin_dict[startpos]
            # -> binidx:
                    # -> sample[samplename]
                            # -> rd averaged
                            # -> baf averaged
                    # -> numsnps
                    # -> cluster 

### Creating bins
bin_width = 250000
chrm_dict2 = dict()
for chrm in chrm_dict1.keys():
    print("processing", chrm)
    chrm_dict2[chrm] = dict()
    bin_dict = chrm_dict2[chrm]
    start_pos = 0
    last_snp_pos = max([int(x) for x in set(chrm_dict1['1'].keys())])
    while start_pos < last_snp_pos: 
        # use start_pos as bin idx
        bin_dict[start_pos] = dict()
        my_bin_dict = bin_dict[start_pos]
        
        # find the snps that fit into this bin
        rv = find_snps(chrm_dict1[chrm], start_pos, bin_width)
        if rv != None:
            cluster, num_snps, sample = rv
            my_bin_dict['cluster'] = cluster
            my_bin_dict['numsnps'] = num_snps
            my_bin_dict['sample'] = sample
        start_pos = start_pos + bin_width
        #break
    #break

processing 1
0
	 125000 {'file_idx': 0, 'cluster': 1, 'samples': {'DCIS': [0.27027, 0.687516], 'INV': [0.0714286, 0.254174]}}
250000
750000
	 875000 {'file_idx': 1, 'cluster': 2, 'samples': {'DCIS': [0.0473251, 3.8713500000000005], 'INV': [0.105169, 0.620511]}}
1000000
1000000
	 1125000 {'file_idx': 2, 'cluster': 3, 'samples': {'DCIS': [0.0314663, 3.22518], 'INV': [0.102459, 0.502589]}}
1250000
1250000
	 1375000 {'file_idx': 3, 'cluster': 4, 'samples': {'DCIS': [0.105023, 2.90909], 'INV': [0.163265, 0.464495]}}
1500000
1500000
	 1625000 {'file_idx': 4, 'cluster': 5, 'samples': {'DCIS': [0.0442963, 2.75532], 'INV': [0.111834, 0.452204]}}
1750000
1750000
	 1875000 {'file_idx': 5, 'cluster': 6, 'samples': {'DCIS': [0.0179691, 2.96922], 'INV': [0.0732436, 0.487712]}}
2000000
2000000
	 2125000 {'file_idx': 6, 'cluster': 7, 'samples': {'DCIS': [0.0, 2.89665], 'INV': [0.0731707, 0.468562]}}
2250000
2250000
	 2375000 {'file_idx': 7, 'cluster': 8, 'samples': {'DCIS': [0.0149665, 2.62328], 'INV'

108750000
	 108875000 {'file_idx': 278, 'cluster': 279, 'samples': {'DCIS': [0.419255, 2.08174], 'INV': [0.436364, 0.567012]}}
109000000
109000000
	 109125000 {'file_idx': 279, 'cluster': 280, 'samples': {'DCIS': [0.421712, 2.29913], 'INV': [0.423645, 0.54814]}}
109250000
109250000
	 109375000 {'file_idx': 280, 'cluster': 281, 'samples': {'DCIS': [0.436407, 2.42592], 'INV': [0.4533, 0.554154]}}
109500000
109500000
	 109625000 {'file_idx': 281, 'cluster': 282, 'samples': {'DCIS': [0.406554, 2.54399], 'INV': [0.443946, 0.629601]}}
109750000
109750000
	 109875000 {'file_idx': 282, 'cluster': 283, 'samples': {'DCIS': [0.389545, 2.43815], 'INV': [0.466063, 0.587966]}}
110000000
110000000
	 110125000 {'file_idx': 283, 'cluster': 284, 'samples': {'DCIS': [0.441115, 2.3794], 'INV': [0.450704, 0.595432]}}
110250000
110250000
	 110375000 {'file_idx': 284, 'cluster': 285, 'samples': {'DCIS': [0.422269, 2.31197], 'INV': [0.441441, 0.598653]}}
110500000
110500000
	 110625000 {'file_idx': 285, 'clus

192000000
	 192125000 {'file_idx': 477, 'cluster': 478, 'samples': {'DCIS': [0.494845, 3.1087099999999994], 'INV': [0.448276, 0.71994]}}
192250000
192500000
	 192625000 {'file_idx': 478, 'cluster': 479, 'samples': {'DCIS': [0.448819, 3.20081], 'INV': [0.382979, 0.682928]}}
192750000
192750000
	 192875000 {'file_idx': 479, 'cluster': 480, 'samples': {'DCIS': [0.366516, 3.1039], 'INV': [0.283333, 0.66224]}}
193000000
193000000
	 193125000 {'file_idx': 480, 'cluster': 481, 'samples': {'DCIS': [0.404175, 3.22623], 'INV': [0.40264, 0.695265]}}
193250000
194000000
	 194125000 {'file_idx': 481, 'cluster': 482, 'samples': {'DCIS': [0.378571, 1.86792], 'INV': [0.345455, 0.421385]}}
194250000
195250000
	 195375000 {'file_idx': 482, 'cluster': 483, 'samples': {'DCIS': [0.142857, 2.08718], 'INV': [0.409091, 0.394958]}}
195500000
196500000
	 196625000 {'file_idx': 483, 'cluster': 484, 'samples': {'DCIS': [0.330275, 1.89747], 'INV': [0.377778, 0.413172]}}
196750000
197250000
	 197375000 {'file_idx':

	 33125000 {'file_idx': 704, 'cluster': 705, 'samples': {'DCIS': [0.39924, 2.32793], 'INV': [0.327586, 0.558569]}}
33250000
33250000
	 33375000 {'file_idx': 705, 'cluster': 706, 'samples': {'DCIS': [0.352538, 2.70769], 'INV': [0.374384, 0.622271]}}
33500000
33500000
	 33625000 {'file_idx': 706, 'cluster': 707, 'samples': {'DCIS': [0.354627, 2.7632799999999995], 'INV': [0.323944, 0.654294]}}
33750000
33750000
	 33875000 {'file_idx': 707, 'cluster': 708, 'samples': {'DCIS': [0.388756, 2.69915], 'INV': [0.396104, 0.634542]}}
34000000
34000000
	 34125000 {'file_idx': 708, 'cluster': 709, 'samples': {'DCIS': [0.216366, 2.52481], 'INV': [0.14841, 0.692884]}}
34250000
36500000
	 36625000 {'file_idx': 709, 'cluster': 710, 'samples': {'DCIS': [0.422705, 2.71581], 'INV': [0.246753, 0.650369]}}
36750000
36750000
	 36875000 {'file_idx': 710, 'cluster': 711, 'samples': {'DCIS': [0.383853, 2.64564], 'INV': [0.373134, 0.610352]}}
37000000
37000000
	 37125000 {'file_idx': 711, 'cluster': 712, 'samples

	 127875000 {'file_idx': 904, 'cluster': 905, 'samples': {'DCIS': [0.0352505, 1.41358], 'INV': [0.121795, 0.383832]}}
128000000
128000000
	 128125000 {'file_idx': 905, 'cluster': 906, 'samples': {'DCIS': [0.0326616, 1.36329], 'INV': [0.113636, 0.35752800000000007]}}
128250000
128250000
	 128375000 {'file_idx': 906, 'cluster': 907, 'samples': {'DCIS': [0.0477999, 1.34481], 'INV': [0.0956989, 0.3600650000000001]}}
128500000
128500000
	 128625000 {'file_idx': 907, 'cluster': 908, 'samples': {'DCIS': [0.0313059, 1.42735], 'INV': [0.120433, 0.395251]}}
128750000
128750000
	 128875000 {'file_idx': 908, 'cluster': 909, 'samples': {'DCIS': [0.0489796, 1.36685], 'INV': [0.103896, 0.365891]}}
129000000
129250000
	 129375000 {'file_idx': 909, 'cluster': 910, 'samples': {'DCIS': [0.00746269, 1.29418], 'INV': [0.0566038, 0.35570500000000005]}}
129500000
130250000
	 130375000 {'file_idx': 910, 'cluster': 911, 'samples': {'DCIS': [0.0333333, 1.42758], 'INV': [0.130435, 0.420613]}}
130500000
130750000

...

86750000
	 86875000 {'file_idx': 5260, 'cluster': 5261, 'samples': {'DCIS': [0.25, 2.25184], 'INV': [0.483333, 0.540517]}}
87000000
88250000
	 88375000 {'file_idx': 5261, 'cluster': 5262, 'samples': {'DCIS': [0.0336134, 1.01765], 'INV': [0.130435, 0.26642]}}
88500000
88500000
	 88625000 {'file_idx': 5262, 'cluster': 5263, 'samples': {'DCIS': [0.0916031, 0.923563], 'INV': [0.175258, 0.278769]}}
88750000
89000000
	 89125000 {'file_idx': 5263, 'cluster': 5264, 'samples': {'DCIS': [0.0380313, 1.04638], 'INV': [0.122137, 0.315459]}}
89250000
89250000
	 89375000 {'file_idx': 5264, 'cluster': 5265, 'samples': {'DCIS': [0.030303, 1.04991], 'INV': [0.142056, 0.324395]}}
89500000
89500000
	 89625000 {'file_idx': 5265, 'cluster': 5266, 'samples': {'DCIS': [0.0264317, 0.962861], 'INV': [0.128571, 0.317864]}}
89750000
89750000
	 89875000 {'file_idx': 5266, 'cluster': 5267, 'samples': {'DCIS': [0.0426357, 1.08192], 'INV': [0.0819672, 0.312506]}}
90000000
90000000
	 90125000 {'file_idx': 5267, 'clust

processing 17
0
	 125000 {'file_idx': 5495, 'cluster': 5496, 'samples': {'DCIS': [0.180602, 2.56169], 'INV': [0.254545, 0.662179]}}
250000
250000
	 375000 {'file_idx': 5496, 'cluster': 5497, 'samples': {'DCIS': [0.0164384, 2.77191], 'INV': [0.109589, 0.719537]}}
500000
500000
	 625000 {'file_idx': 5497, 'cluster': 5498, 'samples': {'DCIS': [0.0173235, 2.57666], 'INV': [0.0902527, 0.653883]}}
750000
750000
	 875000 {'file_idx': 5498, 'cluster': 5499, 'samples': {'DCIS': [0.0295964, 2.71675], 'INV': [0.0566229, 0.672315]}}
1000000
1000000
	 1125000 {'file_idx': 5499, 'cluster': 5500, 'samples': {'DCIS': [0.0138249, 3.04558], 'INV': [0.075, 0.697187]}}
1250000
1250000
	 1375000 {'file_idx': 5500, 'cluster': 5501, 'samples': {'DCIS': [0.0454545, 2.74017], 'INV': [0.0953471, 0.669903]}}
1500000
1500000
	 1625000 {'file_idx': 5501, 'cluster': 5502, 'samples': {'DCIS': [0.0298404, 2.71242], 'INV': [0.0576803, 0.691818]}}
1750000
1750000
	 1875000 {'file_idx': 5502, 'cluster': 5503, 'samples':

66000000
	 66125000 {'file_idx': 5700, 'cluster': 5701, 'samples': {'DCIS': [0.0125523, 2.46717], 'INV': [0.056391, 0.664604]}}
66250000
66250000
	 66375000 {'file_idx': 5701, 'cluster': 5702, 'samples': {'DCIS': [0.017094, 2.81932], 'INV': [0.0884718, 0.703146]}}
66500000
66500000
	 66625000 {'file_idx': 5702, 'cluster': 5703, 'samples': {'DCIS': [0.0167598, 2.56629], 'INV': [0.045614, 0.640657]}}
66750000
66750000
	 66875000 {'file_idx': 5703, 'cluster': 5704, 'samples': {'DCIS': [0.0426309, 2.46019], 'INV': [0.0612245, 0.592234]}}
67000000
67000000
	 67125000 {'file_idx': 5704, 'cluster': 5705, 'samples': {'DCIS': [0.123711, 2.35739], 'INV': [0.226415, 0.569819]}}
67250000
67250000
	 67375000 {'file_idx': 5705, 'cluster': 5706, 'samples': {'DCIS': [0.0180467, 2.43864], 'INV': [0.0722367, 0.5747]}}
67500000
67500000
	 67625000 {'file_idx': 5706, 'cluster': 5707, 'samples': {'DCIS': [0.0216802, 2.72018], 'INV': [0.0848708, 0.640505]}}
67750000
68000000
	 68125000 {'file_idx': 5707, 'c

processing 19
250000
	 375000 {'file_idx': 5900, 'cluster': 5901, 'samples': {'DCIS': [0.0256125, 1.43187], 'INV': [0.155963, 0.421494]}}
500000
500000
	 625000 {'file_idx': 5901, 'cluster': 5902, 'samples': {'DCIS': [0.12101, 1.40032], 'INV': [0.159875, 0.393613]}}
750000
750000
	 875000 {'file_idx': 5902, 'cluster': 5903, 'samples': {'DCIS': [0.0302932, 1.52909], 'INV': [0.136364, 0.413315]}}
1000000
1000000
	 1125000 {'file_idx': 5903, 'cluster': 5904, 'samples': {'DCIS': [0.0241957, 1.43125], 'INV': [0.106452, 0.385026]}}
1250000
1250000
	 1375000 {'file_idx': 5904, 'cluster': 5905, 'samples': {'DCIS': [0.0473996, 1.36456], 'INV': [0.10929, 0.36180999999999996]}}
1500000
1500000
	 1625000 {'file_idx': 5905, 'cluster': 5906, 'samples': {'DCIS': [0.0416667, 1.50424], 'INV': [0.0877193, 0.3917]}}
1750000
1750000
	 1875000 {'file_idx': 5906, 'cluster': 5907, 'samples': {'DCIS': [0.028436, 1.43786], 'INV': [0.123472, 0.399809]}}
2000000
2000000
	 2125000 {'file_idx': 5907, 'cluster': 59

	 41375000 {'file_idx': 6194, 'cluster': 6195, 'samples': {'DCIS': [0.0305556, 1.41818], 'INV': [0.258333, 0.393326]}}
41500000
42000000
	 42125000 {'file_idx': 6195, 'cluster': 6196, 'samples': {'DCIS': [0.0341598, 1.47409], 'INV': [0.108742, 0.388304]}}
42250000
42250000
	 42375000 {'file_idx': 6196, 'cluster': 6197, 'samples': {'DCIS': [0.0367321, 1.51186], 'INV': [0.115207, 0.4066]}}
42500000
42500000
	 42625000 {'file_idx': 6197, 'cluster': 6198, 'samples': {'DCIS': [0.0394089, 1.47942], 'INV': [0.106383, 0.377293]}}
42750000
42750000
	 42875000 {'file_idx': 6198, 'cluster': 6199, 'samples': {'DCIS': [0.029321, 1.45118], 'INV': [0.115942, 0.395835]}}
43000000
43000000
	 43125000 {'file_idx': 6199, 'cluster': 6200, 'samples': {'DCIS': [0.0391459, 1.4071], 'INV': [0.117647, 0.394313]}}
43250000
43250000
	 43375000 {'file_idx': 6200, 'cluster': 6201, 'samples': {'DCIS': [0.0153846, 1.45442], 'INV': [0.2, 0.387567]}}
43500000
43500000
	 43625000 {'file_idx': 6201, 'cluster': 6202, 'sa

In [115]:
chrm_dict2['1']

{0: {'cluster': 1,
  'numsnps': 1,
  'sample': {'INV': {'RDR': 0.254174, 'BAF': 0.0714286},
   'DCIS': {'RDR': 0.687516, 'BAF': 0.27027}}},
 250000: {},
 500000: {},
 750000: {'cluster': 2,
  'numsnps': 1,
  'sample': {'INV': {'RDR': 0.620511, 'BAF': 0.105169},
   'DCIS': {'RDR': 3.8713500000000005, 'BAF': 0.0473251}}},
 1000000: {'cluster': 3,
  'numsnps': 1,
  'sample': {'INV': {'RDR': 0.502589, 'BAF': 0.102459},
   'DCIS': {'RDR': 3.22518, 'BAF': 0.0314663}}},
 1250000: {'cluster': 4,
  'numsnps': 1,
  'sample': {'INV': {'RDR': 0.464495, 'BAF': 0.163265},
   'DCIS': {'RDR': 2.90909, 'BAF': 0.105023}}},
 1500000: {'cluster': 5,
  'numsnps': 1,
  'sample': {'INV': {'RDR': 0.452204, 'BAF': 0.111834},
   'DCIS': {'RDR': 2.75532, 'BAF': 0.0442963}}},
 1750000: {'cluster': 6,
  'numsnps': 1,
  'sample': {'INV': {'RDR': 0.487712, 'BAF': 0.0732436},
   'DCIS': {'RDR': 2.96922, 'BAF': 0.0179691}}},
 2000000: {'cluster': 7,
  'numsnps': 1,
  'sample': {'INV': {'RDR': 0.468562, 'BAF': 0.073170

In [116]:
### Write to file

cov = '-'
alpha = '-'
beta = '-'
u_clone = '-'
cn_clone = '-'
        
with open(output_file, "w+") as w:
    w.write('#CHR\tSTART\tEND\tSAMPLE\tRD\t#SNPS\tCOV\tALPHA\tBETA\tBAF\tCLUSTER\tcn_normal\tu_normal\tcn_clone1\tu_clone1\n')
    for chrm in chrm_dict2.keys():
        for bin_start in chrm_dict2[chrm]:
            mybindict = chrm_dict2[chrm][bin_start]
            
            chrm = str(chrm)
            start = str(bin_start)
            end = str(bin_start + bin_width)
            if not mybindict:
                continue
            cluster = str(mybindict['cluster'])
            num_snps = str(mybindict['numsnps'])
            for sample in mybindict['sample'].keys():
                avg_rdr = str(mybindict['sample'][sample]['RDR'])
                avg_baf = str(mybindict['sample'][sample]['BAF'])
        
                print('chr%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s' % \
                      (chrm, start, end, sample, avg_rdr, num_snps, \
                       cov, alpha, beta, avg_baf, cluster, \
                       u_clone, cn_clone, u_clone, cn_clone))
                w.write('chr%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\n' % (chrm, start, end, sample, rd, num_snps, cov, alpha, beta, baf, cluster, u_clone, cn_clone, u_clone, cn_clone))
        

chr1	0	250000	INV	0.254174	1	-	-	-	0.0714286	1	-	-	-	-
chr1	0	250000	DCIS	0.687516	1	-	-	-	0.27027	1	-	-	-	-
chr1	750000	1000000	INV	0.620511	1	-	-	-	0.105169	2	-	-	-	-
chr1	750000	1000000	DCIS	3.8713500000000005	1	-	-	-	0.0473251	2	-	-	-	-
chr1	1000000	1250000	INV	0.502589	1	-	-	-	0.102459	3	-	-	-	-
chr1	1000000	1250000	DCIS	3.22518	1	-	-	-	0.0314663	3	-	-	-	-
chr1	1250000	1500000	INV	0.464495	1	-	-	-	0.163265	4	-	-	-	-
chr1	1250000	1500000	DCIS	2.90909	1	-	-	-	0.105023	4	-	-	-	-
chr1	1500000	1750000	INV	0.452204	1	-	-	-	0.111834	5	-	-	-	-
chr1	1500000	1750000	DCIS	2.75532	1	-	-	-	0.0442963	5	-	-	-	-
chr1	1750000	2000000	INV	0.487712	1	-	-	-	0.0732436	6	-	-	-	-
chr1	1750000	2000000	DCIS	2.96922	1	-	-	-	0.0179691	6	-	-	-	-
chr1	2000000	2250000	INV	0.468562	1	-	-	-	0.0731707	7	-	-	-	-
chr1	2000000	2250000	DCIS	2.89665	1	-	-	-	0.0	7	-	-	-	-
chr1	2250000	2500000	INV	0.434206	1	-	-	-	0.109665	8	-	-	-	-
chr1	2250000	2500000	DCIS	2.62328	1	-	-	-	0.0149665	8	-	-	-	-
chr1	2500000	2750000	INV	0

chr2	91750000	92000000	INV	0.507276	1	-	-	-	0.389046	832	-	-	-	-
chr2	91750000	92000000	DCIS	2.04055	1	-	-	-	0.346836	832	-	-	-	-
chr2	92000000	92250000	INV	0.488136	1	-	-	-	0.377273	833	-	-	-	-
chr2	92000000	92250000	DCIS	1.74145	1	-	-	-	0.34144	833	-	-	-	-
chr2	92250000	92500000	INV	0.84005	1	-	-	-	0.330393	834	-	-	-	-
chr2	92250000	92500000	DCIS	2.9891499999999995	1	-	-	-	0.344701	834	-	-	-	-
chr2	95250000	95500000	INV	0.518156	1	-	-	-	0.466667	835	-	-	-	-
chr2	95250000	95500000	DCIS	2.22868	1	-	-	-	0.349282	835	-	-	-	-
chr2	95500000	95750000	INV	0.545474	1	-	-	-	0.363426	836	-	-	-	-
chr2	95500000	95750000	DCIS	2.08423	1	-	-	-	0.419503	836	-	-	-	-
chr2	95750000	96000000	INV	0.582734	1	-	-	-	0.382022	837	-	-	-	-
chr2	95750000	96000000	DCIS	2.34877	1	-	-	-	0.434395	837	-	-	-	-
chr2	96000000	96250000	INV	0.555662	1	-	-	-	0.412698	838	-	-	-	-
chr2	96000000	96250000	DCIS	1.8986	1	-	-	-	0.365217	838	-	-	-	-
chr2	96250000	96500000	INV	0.556643	1	-	-	-	0.47619	839	-	-	-	-
chr2	96250000	9650

chr4	39750000	40000000	DCIS	1.96257	1	-	-	-	0.326848	1666	-	-	-	-
chr4	40000000	40250000	INV	0.511453	1	-	-	-	0.340336	1667	-	-	-	-
chr4	40000000	40250000	DCIS	1.9546	1	-	-	-	0.311065	1667	-	-	-	-
chr4	40250000	40500000	INV	0.523532	1	-	-	-	0.402985	1668	-	-	-	-
chr4	40250000	40500000	DCIS	1.95313	1	-	-	-	0.405405	1668	-	-	-	-
chr4	40750000	41000000	INV	0.491917	1	-	-	-	0.359862	1669	-	-	-	-
chr4	40750000	41000000	DCIS	1.88429	1	-	-	-	0.345725	1669	-	-	-	-
chr4	41000000	41250000	INV	0.551877	1	-	-	-	0.368421	1670	-	-	-	-
chr4	41000000	41250000	DCIS	2.00978	1	-	-	-	0.330508	1670	-	-	-	-
chr4	41250000	41500000	INV	0.490701	1	-	-	-	0.352	1671	-	-	-	-
chr4	41250000	41500000	DCIS	2.0985	1	-	-	-	0.326754	1671	-	-	-	-
chr4	41500000	41750000	INV	0.482333	1	-	-	-	0.312139	1672	-	-	-	-
chr4	41500000	41750000	DCIS	1.87921	1	-	-	-	0.263672	1672	-	-	-	-
chr4	41750000	42000000	INV	0.47015	1	-	-	-	0.401515	1673	-	-	-	-
chr4	41750000	42000000	DCIS	1.76967	1	-	-	-	0.316279	1673	-	-	-	-
chr4	42000000	42

chr6	125250000	125500000	INV	0.407639	1	-	-	-	0.389706	2563	-	-	-	-
chr6	125250000	125500000	DCIS	1.58944	1	-	-	-	0.305654	2563	-	-	-	-
chr6	126000000	126250000	INV	0.386631	1	-	-	-	0.381443	2564	-	-	-	-
chr6	126000000	126250000	DCIS	1.71904	1	-	-	-	0.277519	2564	-	-	-	-
chr6	126250000	126500000	INV	0.379934	1	-	-	-	0.443925	2565	-	-	-	-
chr6	126250000	126500000	DCIS	1.62166	1	-	-	-	0.297297	2565	-	-	-	-
chr6	127750000	128000000	INV	0.36896	1	-	-	-	0.406977	2566	-	-	-	-
chr6	127750000	128000000	DCIS	1.53592	1	-	-	-	0.240088	2566	-	-	-	-
chr6	128000000	128250000	INV	0.385736	1	-	-	-	0.357143	2567	-	-	-	-
chr6	128000000	128250000	DCIS	1.66355	1	-	-	-	0.194245	2567	-	-	-	-
chr6	128250000	128500000	INV	0.371667	1	-	-	-	0.426513	2568	-	-	-	-
chr6	128250000	128500000	DCIS	1.5734	1	-	-	-	0.294006	2568	-	-	-	-
chr6	129500000	129750000	INV	0.39852	1	-	-	-	0.382353	2569	-	-	-	-
chr6	129500000	129750000	DCIS	1.63317	1	-	-	-	0.266033	2569	-	-	-	-
chr6	129750000	130000000	INV	0.381815	1	-	-	-	0.423

chr9	65750000	66000000	INV	0.309976	1	-	-	-	0.365079	3470	-	-	-	-
chr9	65750000	66000000	DCIS	0.754157	1	-	-	-	0.239316	3470	-	-	-	-
chr9	66250000	66500000	INV	0.35909299999999994	1	-	-	-	0.266332	3471	-	-	-	-
chr9	66250000	66500000	DCIS	1.25405	1	-	-	-	0.241624	3471	-	-	-	-
chr9	66500000	66750000	INV	0.34291800000000006	1	-	-	-	0.337931	3472	-	-	-	-
chr9	66500000	66750000	DCIS	0.911236	1	-	-	-	0.253687	3472	-	-	-	-
chr9	66750000	67000000	INV	0.491056	1	-	-	-	0.465116	3473	-	-	-	-
chr9	66750000	67000000	DCIS	1.50596	1	-	-	-	0.378601	3473	-	-	-	-
chr9	67000000	67250000	INV	0.325243	1	-	-	-	0.48	3474	-	-	-	-
chr9	67000000	67250000	DCIS	1.06634	1	-	-	-	0.331126	3474	-	-	-	-
chr9	67250000	67500000	INV	0.386553	1	-	-	-	0.362424	3475	-	-	-	-
chr9	67250000	67500000	DCIS	1.19457	1	-	-	-	0.376412	3475	-	-	-	-
chr9	67750000	68000000	INV	0.315733	1	-	-	-	0.356322	3476	-	-	-	-
chr9	67750000	68000000	DCIS	1.18173	1	-	-	-	0.320276	3476	-	-	-	-
chr9	68250000	68500000	INV	0.328047	1	-	-	-	0.354928	347

chr11	118250000	118500000	DCIS	2.7885	1	-	-	-	0.456311	4312	-	-	-	-
chr11	118500000	118750000	INV	0.600061	1	-	-	-	0.385787	4313	-	-	-	-
chr11	118500000	118750000	DCIS	2.82354	1	-	-	-	0.378562	4313	-	-	-	-
chr11	118750000	119000000	INV	0.611733	1	-	-	-	0.42913	4314	-	-	-	-
chr11	118750000	119000000	DCIS	2.86403	1	-	-	-	0.452779	4314	-	-	-	-
chr11	119000000	119250000	INV	0.60096	1	-	-	-	0.452605	4315	-	-	-	-
chr11	119000000	119250000	DCIS	2.7988800000000005	1	-	-	-	0.429239	4315	-	-	-	-
chr11	119250000	119500000	INV	0.571739	1	-	-	-	0.45098	4316	-	-	-	-
chr11	119250000	119500000	DCIS	2.69932	1	-	-	-	0.458537	4316	-	-	-	-
chr11	119500000	119750000	INV	0.594839	1	-	-	-	0.407407	4317	-	-	-	-
chr11	119500000	119750000	DCIS	2.86936	1	-	-	-	0.448276	4317	-	-	-	-
chr11	119750000	120000000	INV	0.517771	1	-	-	-	0.388489	4318	-	-	-	-
chr11	119750000	120000000	DCIS	2.71677	1	-	-	-	0.308943	4318	-	-	-	-
chr11	120000000	120250000	INV	0.564404	1	-	-	-	0.435185	4319	-	-	-	-
chr11	120000000	120250000	D

chr15	100250000	100500000	INV	0.483709	1	-	-	-	0.0676923	5291	-	-	-	-
chr15	100250000	100500000	DCIS	1.64152	1	-	-	-	0.0242165	5291	-	-	-	-
chr15	100500000	100750000	INV	0.506247	1	-	-	-	0.086771	5292	-	-	-	-
chr15	100500000	100750000	DCIS	1.73167	1	-	-	-	0.0237213	5292	-	-	-	-
chr15	100750000	101000000	INV	0.468936	1	-	-	-	0.0769231	5293	-	-	-	-
chr15	100750000	101000000	DCIS	1.74456	1	-	-	-	0.0291971	5293	-	-	-	-
chr15	101000000	101250000	INV	0.482454	1	-	-	-	0.0547445	5294	-	-	-	-
chr15	101000000	101250000	DCIS	1.65569	1	-	-	-	0.0364465	5294	-	-	-	-
chr15	101250000	101500000	INV	0.513795	1	-	-	-	0.0	5295	-	-	-	-
chr15	101250000	101500000	DCIS	1.61722	1	-	-	-	0.111111	5295	-	-	-	-
chr15	101500000	101750000	INV	0.508469	1	-	-	-	0.100551	5296	-	-	-	-
chr15	101500000	101750000	DCIS	1.77791	1	-	-	-	0.0243478	5296	-	-	-	-
chr15	101750000	102000000	INV	0.501478	1	-	-	-	0.0977702	5297	-	-	-	-
chr15	101750000	102000000	DCIS	1.76536	1	-	-	-	0.0421601	5297	-	-	-	-
chr15	102250000	102500000	INV

chr19	50250000	50500000	INV	0.597256	1	-	-	-	0.416712	6062	-	-	-	-
chr19	50250000	50500000	DCIS	2.24623	1	-	-	-	0.420444	6062	-	-	-	-
chr19	50500000	50750000	INV	0.68631	1	-	-	-	0.437558	6063	-	-	-	-
chr19	50500000	50750000	DCIS	2.37146	1	-	-	-	0.435403	6063	-	-	-	-
chr19	50750000	51000000	INV	0.659558	1	-	-	-	0.4	6064	-	-	-	-
chr19	50750000	51000000	DCIS	2.3239	1	-	-	-	0.393109	6064	-	-	-	-
chr19	51000000	51250000	INV	0.65113	1	-	-	-	0.409091	6065	-	-	-	-
chr19	51000000	51250000	DCIS	2.32304	1	-	-	-	0.400871	6065	-	-	-	-
chr19	51250000	51500000	INV	0.656308	1	-	-	-	0.445571	6066	-	-	-	-
chr19	51250000	51500000	DCIS	2.3222	1	-	-	-	0.415132	6066	-	-	-	-
chr19	51500000	51750000	INV	0.627261	1	-	-	-	0.436193	6067	-	-	-	-
chr19	51500000	51750000	DCIS	2.18675	1	-	-	-	0.42277	6067	-	-	-	-
chr19	51750000	52000000	INV	0.617671	1	-	-	-	0.454248	6068	-	-	-	-
chr19	51750000	52000000	DCIS	2.18819	1	-	-	-	0.374121	6068	-	-	-	-
chr19	52000000	52250000	INV	0.590528	1	-	-	-	0.450153	6069	-	-	-	-
chr19